# JC2BIM 2021: TP Burrows-Wheeler Transform
>Téo Lemane, Claire Lemaitre

In [2]:
from typing import List, Dict, Tuple, Callable
from utils import (
    print_table,    # Pretty-printing. Usage: print_table(text, suffix_array, bwt, rank)
    naive_matching, # Naive pattern matching, for comparison. Usage: naive_matching(pattern, text)
    timeit          # Function timer. Usage: timeit(func, *args)
)

## 1. Construction

### 1.1 Suffix Array

**Q1)** Write a function `suffix_array(seq, term)` which returns the suffix array of `seq`. 

`term` is the character symbol indicating the end of the sequence, do not forget to add it at the end of your sequence, this is crucial for the BWT. 

(Hints: [sorted](https://docs.python.org/3/howto/sorting.html#sorting-basics), [key](https://docs.python.org/3/howto/sorting.html#key-functions), [lambda](https://docs.python.org/3.10/reference/expressions.html?#lambda))

In [3]:
sa_t = List[int] # suffix array is list of integers

def suffix_array(seq: str, term: str="$") -> sa_t:
    if seq[-1] != term:
        seq += term
    # init sa with all suffix positions
    sa = [x for x in range(len(seq))]
    # sorting sa according to lexicographical order of the suffixes
    sorted_sa = sorted(sa, key=lambda i: seq[i:])
    return(sorted_sa)

In [4]:
from data import (
    seq_test, # test sequence
    sa_test   # expected suffix array
)
print(seq_test)
sa = suffix_array(seq_test)
print(sa)
print_table(seq_test+"$", sa_test, None, None)

ACGACTACAGACAT
[14, 6, 10, 0, 3, 8, 12, 7, 11, 1, 4, 9, 2, 13, 5]
i   SA[i]  S[SA[i]:]        F  L  Rank  
0   14     $                $           
1   6      ACAGACAT$        A           
2   10     ACAT$            A           
3   0      ACGACTACAGACAT$  A           
4   3      ACTACAGACAT$     A           
5   8      AGACAT$          A           
6   12     AT$              A           
7   7      CAGACAT$         C           
8   11     CAT$             C           
9   1      CGACTACAGACAT$   C           
10  4      CTACAGACAT$      C           
11  9      GACAT$           G           
12  2      GACTACAGACAT$    G           
13  13     T$               T           
14  5      TACAGACAT$       T           


### 1.2 BWT

Reminder :

$BWT_S[i] = \left\{ 
    \begin{array}{rll}
         S[SA[i]-1] & \mbox{if}
         & SA[i]>0 \\ '\$'  & \mbox{if} & SA[i]=0
    \end{array}\right.$

**Q2)** Write a function `bwt(seq, sa)` which returns the bwt of `seq`.

In [5]:
bwt_t = List[str] # bwt is a list of characters

def bwt(seq: str, sa: sa_t) -> bwt_t:
    bwt = []
    for i in range(len(sa)):
        pos = sa[i]
        if pos == 0:
            bwt.append("$")
        else:
            bwt.append(seq[pos-1])
    return(bwt)
def bwt2(seq: str, sa: sa_t) -> bwt_t:
    return [seq[i - 1] if i > 0 else "$" for i in sa]

In [6]:
print(bwt(seq_test,sa))
print(bwt2(seq_test,sa))
from data import bwt_test # expected bwt of seq_test
print_table(seq_test+"$", sa_test, bwt_test, None)

['T', 'T', 'G', '$', 'G', 'C', 'C', 'A', 'A', 'A', 'A', 'A', 'C', 'A', 'C']
['T', 'T', 'G', '$', 'G', 'C', 'C', 'A', 'A', 'A', 'A', 'A', 'C', 'A', 'C']
i   SA[i]  S[SA[i]:]        F  L  Rank  
0   14     $                $  T        
1   6      ACAGACAT$        A  T        
2   10     ACAT$            A  G        
3   0      ACGACTACAGACAT$  A  $        
4   3      ACTACAGACAT$     A  G        
5   8      AGACAT$          A  C        
6   12     AT$              A  C        
7   7      CAGACAT$         C  A        
8   11     CAT$             C  A        
9   1      CGACTACAGACAT$   C  A        
10  4      CTACAGACAT$      C  A        
11  9      GACAT$           G  A        
12  2      GACTACAGACAT$    G  C        
13  13     T$               T  A        
14  5      TACAGACAT$       T  C        


### 1.3 FM-index : BWT + 2 tables (rank and occ) and LF mapping function

**Q3)** Write a function `rank_occ(bwt, lexi)` which returns the rank array and the occurence table.

**Example:**  
BWT = $T_0T_1G_0\$_0G_1C_0C_1A_0A_1A_2A_3A_4C_2A_5C_3$  
rank = [0, 1, 0, 0, 1, 0, 1, 0, 1, 2, 3, 4, 2, 5, 3]  
occ = {'$': 1, 'A': 6, 'C': 4, 'G': 2, 'T': 2}

In [7]:
rank_t = List[int]     # rank is a list of integers 
occ_t = Dict[str, int] # occ is a dictionary

def rank_occ(bwt: bwt_t, lexi: str="$ACGT") -> Tuple[rank_t, occ_t]:
    occ = {c: 0 for c in lexi}
    rank = [0] * len(bwt)
    for i in range(len(bwt)):
        c = bwt[i]
        rank[i] = occ[c]
        occ[c] +=1
    return rank, occ

In [8]:
print(rank_occ(bwt_test))
from data import (
    rank_test, # expected rank of bwt_test
    occ_test   # expected occ of bwt_test
)
print_table(seq_test+"$", sa_test, bwt_test, rank_test)

([0, 1, 0, 0, 1, 0, 1, 0, 1, 2, 3, 4, 2, 5, 3], {'$': 1, 'A': 6, 'C': 4, 'G': 2, 'T': 2})
i   SA[i]  S[SA[i]:]        F  L  Rank  
0   14     $                $  T  0     
1   6      ACAGACAT$        A  T  1     
2   10     ACAT$            A  G  0     
3   0      ACGACTACAGACAT$  A  $  0     
4   3      ACTACAGACAT$     A  G  1     
5   8      AGACAT$          A  C  0     
6   12     AT$              A  C  1     
7   7      CAGACAT$         C  A  0     
8   11     CAT$             C  A  1     
9   1      CGACTACAGACAT$   C  A  2     
10  4      CTACAGACAT$      C  A  3     
11  9      GACAT$           G  A  4     
12  2      GACTACAGACAT$    G  C  2     
13  13     T$               T  A  5     
14  5      TACAGACAT$       T  C  3     


**Q4)** Write a function `LF(c, i, occ, lexi)` which returns the position of $c_i$ in $F$. $c_i$ is the (i+1)-th character of type $c$ in $L$ (BWT).

*In the example above, to apply LF on $G_1$ at position 4 in BWT : c = 'G' and i = 1. (this is the second 'G' of BWT)* 

In [9]:
def LF(c: str, i: int, occ: occ_t, lexi: str="$ACGT") -> int:
    res = 0
    for nt in lexi:
        if c>nt:
            res += occ[nt]
        else:
            return res + i

LF("G",0,occ_test)

11

**Q5)** Using the LF-mapping property, write a function `reverse(bwt, rank, occ)` which returns the original sequence from its BWT.

In [10]:
def reverse(bwt: bwt_t, rank: rank_t, occ: occ_t) -> str:
    seq = ""
    i = 0
    while bwt[i] != '$':
        c = bwt[i]
        seq +=c
        i = LF(c,rank[i],occ)
    return(seq[::-1])
print(seq_test)
reverse(bwt_test,rank_test,occ_test)

ACGACTACAGACAT


'ACGACTACAGACAT'

## 2. Pattern matching with the FM-index

Reminder: to determine if a given word is present in a text indexed with the FM-index, we start from the last character of the word and we use a similar approach as in the function `reverse` but instead of "tracing" one position of the text, we have to trace several positions simultaneously, they are all grouped in an interval. 

Thus, we will loop over the characters of the word, from end to beginning, and update at each step the two indices that delimit the interval on the BWT. At the end (or during the iterations if the interval is empty), the size of the interval gives the number of occurrences of the pattern in the text.

### 2.1 Useful functions
**Q6.1)** Write a function `find_first(c, i, j, bwt)` which returns the position of the first character `c` in $BWT[i,j]$.  
**Q6.2)** Write a function `find_last(c, j, i, bwt)` which returns the position of the last character `c` in $BWT[i,j]$.

In [11]:
def find_first(c: str, i: int, j: int, bwt: bwt_t) -> int:
    pass

def find_last(c: str, j: int, i: int, bwt: bwt_t) -> int:
    pass

### 2.2 Pattern matching function
**Q7)** Write a function `backward_search(pattern, bwt, rank, occ)` which returns `True` if `pattern` is found, `False` otherwise.

In [12]:
def backward_search(pattern: str, bwt: bwt_t, rank: rank_t, occ: occ_t) -> bool:
    pass

## 3. Application to real data


In [14]:
from data import seq, seq_queries

`seq` is a real DNA sequence of size 29,903 bp (the sars-cov-2 genome).

`seq_queries` is a list of 11 20-mers.

We want to determine which ones of these 20-mers appear in the sars-cov-2 genome.

**Q8)** Apply your indexing data structure and the pattern matching function to query all the kmers of `seq_queries` in the sequence `seq`.

In [16]:
_sa = suffix_array(seq)
_bwt = bwt(seq, _sa)
_rank, _occ = rank_occ(_bwt)
# quick test
_reverse_bwt = reverse(_bwt,_rank,_occ)
seq == _reverse_bwt

True

In [17]:
# pattern matching :
for pat in seq_queries:
    print(f"pattern {pat} found in seq ? {backward_search(pat,_bwt,_rank,_occ)}")

pattern CAAAACATTCCCACCAACAG found in seq ? True
pattern TCGGGCACGTAGTGTAGCTA found in seq ? False
pattern GGCTCAATGTGTCCAGTTAC found in seq ? True
pattern GGGAAATTGTTAAATTTATC found in seq ? True
pattern AAAACAAGATGATAAGAAAA found in seq ? True
pattern TTTAGCTTAGTTGCAGAGTG found in seq ? False
pattern TCTTAGCCTACTGTAATAAG found in seq ? True
pattern TTTGAACTTGATGAAAGGAT found in seq ? True
pattern CCAGGTAACAAACCAACCAA found in seq ? True
pattern TGCAGGAATCTTTTGTTGCA found in seq ? False
pattern AAGACATGTACGTGCATGGA found in seq ? True


**Q9)** Perform the same queries using the function `naive_matching(pattern, text)`, and compare the results and running times.

In [19]:
# pattern matching :
for pat in seq_queries:
    print(f"pattern {pat} found in seq ? bwt = {backward_search(pat,_bwt,_rank,_occ)} naive = {naive_matching(pat, seq)}")

pattern CAAAACATTCCCACCAACAG found in seq ? bwt = True naive = True
pattern TCGGGCACGTAGTGTAGCTA found in seq ? bwt = False naive = False
pattern GGCTCAATGTGTCCAGTTAC found in seq ? bwt = True naive = True
pattern GGGAAATTGTTAAATTTATC found in seq ? bwt = True naive = True
pattern AAAACAAGATGATAAGAAAA found in seq ? bwt = True naive = True
pattern TTTAGCTTAGTTGCAGAGTG found in seq ? bwt = False naive = False
pattern TCTTAGCCTACTGTAATAAG found in seq ? bwt = True naive = True
pattern TTTGAACTTGATGAAAGGAT found in seq ? bwt = True naive = True
pattern CCAGGTAACAAACCAACCAA found in seq ? bwt = True naive = True
pattern TGCAGGAATCTTTTGTTGCA found in seq ? bwt = False naive = False
pattern AAGACATGTACGTGCATGGA found in seq ? bwt = True naive = True


If you need to check the correctness of your FM-index for the sequence `seq`...

In [ ]:
from data import sa_seq, bwt_seq, rank_seq, occ_seq